In [1]:
from common.data_pipeline.dataset import DatasetChainer
from common.data_pipeline.MMCBNU_6000.dataset import DatasetLoader as mmcbnu
from common.data_pipeline.FV_USM.dataset import DatasetLoader as fvusm
from common.util.enums import EnvironmentType

datasets = DatasetChainer(datasets=[mmcbnu(included_portion=0.001), fvusm(included_portion=0)])
train, test, validation = datasets.get_dataset(
    EnvironmentType.PYTORCH,
)

2023-08-22 13:57:01,252 - root - INFO - Compiled train/test/validation split for MMCBNU6000
2023-08-22 13:57:01,328 - root - INFO - Compiled train/test/validation split for FV_USM
2023-08-22 13:57:01,328 - root - INFO - Concatenating train set
2023-08-22 13:57:01,329 - root - INFO - Concatenating test set
2023-08-22 13:57:01,330 - root - INFO - Concatenating validation set


In [2]:
from common.util.enums import DatasetSplitType


datasets.get_files(DatasetSplitType.TRAIN)

[DatasetObject(path='./datasets/MMCBNU_6000/ROIs/028/L_Ring/08.bmp', name='028/L_Ring/08.bmp', label=None, mask_path=None, metadata={'finger': 'ring', 'hand': 'left', 'is_augmented': False}),
 DatasetObject(path='./datasets/MMCBNU_6000/ROIs/096/R_Ring/03.bmp', name='096/R_Ring/03.bmp', label=None, mask_path=None, metadata={'finger': 'ring', 'hand': 'right', 'is_augmented': False}),
 DatasetObject(path='./datasets/MMCBNU_6000/ROIs/089/L_Ring/05.bmp', name='089/L_Ring/05.bmp', label=None, mask_path=None, metadata={'finger': 'ring', 'hand': 'left', 'is_augmented': False}),
 DatasetObject(path='./datasets/MMCBNU_6000/ROIs/064/L_Ring/06.bmp', name='064/L_Ring/06.bmp', label=None, mask_path=None, metadata={'finger': 'ring', 'hand': 'left', 'is_augmented': False})]